In [32]:
import json 
import csv
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.tokenize import word_tokenize
import nltk
from transformers import BertTokenizer
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, Lambda
from sklearn.model_selection import train_test_split
from transformers import TFBertModel
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [9]:
def tsv2json(input_file, output_file):
    arr = []
    
    with open(input_file, 'r', encoding='utf-8') as file:
        for line in file:
            items = line.split('\t')
            d = {}
            for item in items:
                key, value = item.split('=', 1)
                d[key.strip()] = value.strip()
            arr.append(d)
    
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(arr, file, ensure_ascii=False, indent=4)

In [10]:
input_filename_tskv = 'geo-reviews-dataset-2023.tskv'
output_filename_json = 'geo-reviews-dataset-2023.json'
tsv2json(input_filename_tskv,output_filename_json)

In [11]:
input_filename_json = 'geo-reviews-dataset-2023.json'
df = pd.read_json(input_filename_json, encoding='utf-8')
df_restaurants = df[df['rubrics'].str.contains('Ресторан', case=False)]
output_file_pkl = 'restaurants_reviews.pkl'
df_restaurants.to_pickle(output_file_pkl)

In [12]:
input_file_pkl = 'restaurants_reviews.pkl'
df_restaurants = pd.read_pickle(input_file_pkl)
df_restaurants

,address,name_ru,rating,rubrics,text
13,"Краснодарский край, городской округ Сочи, посё...",Пандок,2,Ресторан,"Самый большой плюс это месторасположение, набе..."
14,"Краснодарский край, городской округ Сочи, посё...",Пандок,5,Ресторан,Добрый день! Сегодня во второй раз посетили с ...
23,"Московская область, Орехово-Зуевский городской...",Пицца РИО,5,Кафе;Ресторан;Столовая,Очень вкусно! И по домашнему! Замечательный ко...
68,"Омск, улица 70 лет Октября, 19",Корова на бочке,5,"Ресторан;Бар, паб","Прекрасное заведение, хожу много лет, меню не ..."
137,"Ярославль, площадь Труда, 1",IZI kitchen,3,Ресторан;Кафе,Отлично местоположение. Еда ничего особенного....
...,...,...,...,...,...
499799,"Республика Крым, Ялта, набережная имени В.И. Л...",Hispaniola Lounge,5,Ресторан,"Безумно понравилось это заведение, до этого ча..."
499817,"Ставрополь, улица Дзержинского, 160",Клюква,5,Ресторан,Мне очень понравилось!\n\nЕдинственный минус -...
499988,"Москва, улица Юных Ленинцев, 52",Petruccio. Pizza&Pasta,5,Ресторан;Кафе;Пиццерия,Очень вкусная пицца. Нравится их система лояль...
499989,"Москва, улица Юных Ленинцев, 52",Petruccio. Pizza&Pasta,5,Ресторан;Кафе;Пиццерия,"Пицца детям понравилась, интерьер очень интере..."


In [13]:
df_restaurants
def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'[^\w\s]', '', text)  
    text = text.lower()  
    
    return text



In [14]:
df_restaurants['text'] = df_restaurants['text'].apply(clean_text)
df_restaurants.drop_duplicates(subset=['text'], inplace=True)
df_restaurants.to_pickle(output_file_pkl)


C:\Users\Alex\AppData\Local\Temp\ipykernel_1428\79882020.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()  # Удаление HTML-тегов


In [15]:
df_restaurants.reset_index(drop=True, inplace=True)
nltk.download('punkt')
def tokenize_text(text):
    return word_tokenize(text)
df_restaurants['tokens'] = df_restaurants['text'].apply(tokenize_text)
print(df_restaurants[['text', 'tokens']].head())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                                text  \
0  самый большой плюс это месторасположение набер...   
1  добрый день сегодня во второй раз посетили с д...   
2  очень вкусно и по домашнему замечательный комп...   
3  прекрасное заведение хожу много лет меню не по...   
4  отлично местоположение еда ничего особенногоnп...   

                                              tokens  
0  [самый, большой, плюс, это, месторасположение,...  
1  [добрый, день, сегодня, во, второй, раз, посет...  
2  [очень, вкусно, и, по, домашнему, замечательны...  
3  [прекрасное, заведение, хожу, много, лет, меню...  
4  [отлично, местоположение, еда, ничего, особенн...  


In [16]:
def rating_to_label(rating):
    if rating <= 2:
        return 0  
    elif rating >= 4:
        return 1  
    else:
        return None 
df_restaurants['label'] = df_restaurants['rating'].apply(rating_to_label)
df_restaurants = df_restaurants.dropna(subset=['label'])

In [18]:
texts = df_restaurants['text'].tolist()
labels = df_restaurants['label'].tolist()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokens = tokenizer(texts, padding=True, truncation=True, max_length=100, return_tensors="tf")
input_ids = tokens['input_ids']
attention_masks = tokens['attention_mask']

input_ids_np = input_ids.numpy()
attention_masks_np = attention_masks.numpy()

train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    input_ids_np, labels, test_size=0.2, random_state=42)
train_masks, test_masks = train_test_split(
    attention_masks_np, test_size=0.2, random_state=42)

train_inputs = tf.convert_to_tensor(train_inputs)
test_inputs = tf.convert_to_tensor(test_inputs)
train_masks = tf.convert_to_tensor(train_masks)
test_masks = tf.convert_to_tensor(test_masks)
train_labels = tf.convert_to_tensor(train_labels)
test_labels = tf.convert_to_tensor(test_labels)

In [30]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

input_ids = Input(shape=(100,), dtype=tf.int32, name='input_ids')
attention_mask = Input(shape=(100,), dtype=tf.int32, name='attention_mask')

def bert_layer(inputs):
    input_ids, attention_mask = inputs
    return bert_model(input_ids=input_ids, attention_mask=attention_mask)[0]

bert_output = Lambda(bert_layer, output_shape=(100, 768))([input_ids, attention_mask]) 
x = GlobalAveragePooling1D()(bert_output)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[input_ids, attention_mask], outputs=output)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(
    [train_inputs, train_masks],
    train_labels,
    validation_data=([test_inputs, test_masks], test_labels),
    epochs=3,
    batch_size=16
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/3
  28/2686 ━━━━━━━━━━━━━━━━━━━━ 26:21 595ms/step - accuracy: 0.9097 - loss: 0.3847

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate([test_inputs, test_masks], test_labels)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [33]:
engine = create_engine('sqlite:///C:/Users/Alex/Documents/projects/restaurants_reviews/reviews.db')

Base = declarative_base()

class ClassificationResult(Base):
    __tablename__ = 'classification_results'

    id = Column(Integer, primary_key=True)
    review_text = Column(String)
    predicted_rating = Column(Float)

Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

C:\Users\Alex\AppData\Local\Temp\ipykernel_1428\540592378.py:4: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
